In [ ]:
import snowflake.snowpark as snp
from snowflake.snowpark import functions as F

## Data Masking
We don't want all the users to see the age of our squirrels. We want only the internal devs to access that sensitive data.

Therefore we are going to create a masking policy to dynamically protect this PII column

### 1. Load credentials and connect to Snowflake

In [ ]:
from steps.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect('./include/state.json')
session.use_warehouse(state_dict['compute_parameters']['default_warehouse'])

In [ ]:
session.table(state_dict['table_name']).count()

### 2. Create masking policy to protect our PII data

In [ ]:
try:
  session.sql("CREATE OR REPLACE MASKING POLICY mask_string_age AS \
      (val STRING) RETURNS STRING -> \
      CASE \
      WHEN CURRENT_ROLE() in ('INTERNAL_USER') THEN val \
        ELSE '**masked**'\
      END;").collect()
except:
    pass

Apply the masking policy to table

In [ ]:
session.sql("ALTER TABLE "+ state_dict['table_name'] +" MODIFY COLUMN AGE SET MASKING POLICY mask_string_age;").collect()

In [ ]:
session.sql("grant usage on database "+ state_dict['connection_parameters']['database'] +" to role PUBLIC;").collect()
session.sql("grant usage on schema " + state_dict['connection_parameters']['database'] + "."+ state_dict['connection_parameters']['schema']+ " to role PUBLIC;").collect()
session.sql("grant select on table "+ state_dict['connection_parameters']['database'] + "."+ state_dict['connection_parameters']['schema']+ "." +  state_dict['table_name'] + " to role PUBLIC;").collect()


See the masking policy in action. And a bit of Snowpark magic

In [ ]:
session.use_role(state_dict['additional_roles']['default_role'])
session.table(state_dict['table_name']).select(F.col("age")).limit(5).show()

In [ ]:
session.use_role(state_dict['additional_roles']['project_role'])
session.table(state_dict['table_name']).select(F.col("age")).limit(5).show()

### 3. (Optional) Behind the scenes

In [ ]:
# (optional) behind the scenes
session.sql("SELECT query_type,query_text "+
            "FROM TABLE(information_schema.query_history()) "+
            "ORDER BY start_time DESC "+
            "LIMIT 2").show(max_width=200)

In [ ]:
session.close()